# Limpieza y estructurado de datos del tiempo

Objetivo: unir todos los csv de datos del tiempo en uno solo. Estos datos ya están filtrados para la estación meteorológica del retiro, pero hay que filtrar los valores recogidos que nos interesan. El resultado se guarda en *./tiempo/\<estacion\>*.

> Aclaración: los comentarios incluidos pertenecen a los datos del retiro, aunque el notebook se puede usar con cualquier estación meteorológica.

In [29]:
inPath = "../0 data/tiempo/"
outPath = "./tiempo/"

estacion = "Getafe"

## Unión de datasets

In [30]:
import pandas as pd

In [31]:
# Leer los archivos json que empiezan con "tiempo" en la carpeta inPath
# y juntarlos en un solo DataFrame

import os
files = os.listdir(inPath)
files = [f for f in files if f.startswith(estacion)]

dataframes = []
for f in files:
    dataframes.append(pd.read_json(inPath + f))

df = pd.concat(dataframes, ignore_index=True)
df.tail()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,sol,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
4743,2022-12-27,3200,GETAFE,MADRID,620,"10,1","0,0","3,8",07:20,"16,4",...,"8,8","960,1",10,"956,7",24,84.0,NaN,NaN,NaN,NaN
4744,2022-12-28,3200,GETAFE,MADRID,620,"8,8","0,0","3,0",08:00,"14,5",...,"8,6","956,7",00,"951,5",16,85.0,NaN,NaN,NaN,NaN
4745,2022-12-29,3200,GETAFE,MADRID,620,"9,7",Ip,"6,8",Varias,"12,6",...,"0,9","951,9",11,"949,8",16,88.0,NaN,NaN,NaN,NaN
4746,2022-12-30,3200,GETAFE,MADRID,620,"10,4","0,0","6,4",23:59,"14,5",...,"2,8","955,1",24,"949,7",04,66.0,NaN,NaN,NaN,NaN
4747,2022-12-31,3200,GETAFE,MADRID,620,"8,7","0,0","3,0",06:30,"14,4",...,"3,7","955,4",10,"952,9",17,73.0,NaN,NaN,NaN,NaN


In [32]:
# Inspeccionar los datos
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fecha        4748 non-null   object 
 1   indicativo   4748 non-null   int64  
 2   nombre       4748 non-null   object 
 3   provincia    4748 non-null   object 
 4   altitud      4748 non-null   int64  
 5   tmed         4748 non-null   object 
 6   prec         4748 non-null   object 
 7   tmin         4748 non-null   object 
 8   horatmin     4748 non-null   object 
 9   tmax         4748 non-null   object 
 10  horatmax     4748 non-null   object 
 11  dir          4748 non-null   int64  
 12  velmedia     4748 non-null   object 
 13  racha        4748 non-null   object 
 14  horaracha    4747 non-null   object 
 15  sol          4748 non-null   object 
 16  presMax      4748 non-null   object 
 17  horaPresMax  4748 non-null   object 
 18  presMin      4748 non-null   object 
 19  horaPr

,indicativo,altitud,dir,hrMedia,hrMax,hrMin
count,4748.0,4748.0,4748.000000,4745.000000,44.000000,43.000000
mean,3200.0,620.0,21.946504,55.051001,79.272727,32.465116
std,0.0,0.0,16.371750,19.648351,17.750681,16.379937
min,3200.0,620.0,1.000000,14.000000,46.000000,10.000000
25%,3200.0,620.0,10.000000,38.000000,61.750000,18.500000
50%,3200.0,620.0,23.000000,53.000000,86.000000,28.000000
75%,3200.0,620.0,28.000000,71.000000,94.250000,45.500000
max,3200.0,620.0,99.000000,100.000000,100.000000,65.000000


## Filtrado a columnas que nos interesan por estar también en la predicción

> Columnas: fecha, prec, tmin, tmax, dir, velmedia, racha

In [33]:
# Quedarse con las columnas que nos interesan
# Columnas: fecha, prec, tmin, tmax, dir, velmedia, racha, sol
df = df[["fecha", "prec", "tmin", "tmax", "dir", "velmedia", "racha", "sol"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fecha     4748 non-null   object
 1   prec      4748 non-null   object
 2   tmin      4748 non-null   object
 3   tmax      4748 non-null   object
 4   dir       4748 non-null   int64 
 5   velmedia  4748 non-null   object
 6   racha     4748 non-null   object
 7   sol       4748 non-null   object
dtypes: int64(1), object(7)
memory usage: 296.9+ KB


In [34]:
df.tail()

,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4743,2022-12-27,"0,0","3,8","16,4",21,"0,8","3,6","8,8"
4744,2022-12-28,"0,0","3,0","14,5",26,"1,7","7,8","8,6"
4745,2022-12-29,Ip,"6,8","12,6",27,"4,4","13,3","0,9"
4746,2022-12-30,"0,0","6,4","14,5",27,"6,1","17,5","2,8"
4747,2022-12-31,"0,0","3,0","14,4",16,"1,7","4,2","3,7"


## Limpieza y adaptación de los datos

 - Tipos numéricos

In [35]:
# Cambiar Ip por 0 en la columna prec
df["prec"] = df["prec"].replace("Ip", 0)

# Cambiar 99 y 88 por NaN en la columna dir
df["dir"] = df["dir"].replace(99, None)
df["dir"] = df["dir"].replace(88, None)

# Formatear las columnas a los tipos de datos correctos
df["prec"] = df["prec"].astype(str).str.replace(",", ".").astype(float)
df["tmin"] = df["tmin"].astype(str).str.replace(",", ".").astype(float)
df["tmax"] = df["tmax"].astype(str).str.replace(",", ".").astype(float)
df["dir"] = df["dir"].astype(float)
df["velmedia"] = df["velmedia"].astype(str).str.replace(",", ".").astype(float)
df["racha"] = df["racha"].astype(str).str.replace(",", ".").astype(float)
df["sol"] = df["sol"].astype(str).str.replace(",", ".").astype(float)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     4748 non-null   object 
 1   prec      4748 non-null   float64
 2   tmin      4748 non-null   float64
 3   tmax      4748 non-null   float64
 4   dir       4618 non-null   float64
 5   velmedia  4748 non-null   float64
 6   racha     4748 non-null   float64
 7   sol       4748 non-null   float64
dtypes: float64(7), object(1)
memory usage: 296.9+ KB


In [36]:
df.tail()

,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4743,2022-12-27,0.0,3.8,16.4,21.0,0.8,3.6,8.8
4744,2022-12-28,0.0,3.0,14.5,26.0,1.7,7.8,8.6
4745,2022-12-29,0.0,6.8,12.6,27.0,4.4,13.3,0.9
4746,2022-12-30,0.0,6.4,14.5,27.0,6.1,17.5,2.8
4747,2022-12-31,0.0,3.0,14.4,16.0,1.7,4.2,3.7


 - Se observa que los datos de viento están incompletos (pero son importantes)
 Existe la posibilidad de emplear otra estación meteorológica (aeropuerto de Barajas pej)

 - También se encuentran datos incompletos en las temperaturas, aunque son pocos (se decide prolongar el día anterior), y en las precipitaciones, aunque son menos aún (se decide sustituir por 0)

In [37]:
# Show 5 of november 2022
df[df["fecha"] == "2022-11-05"]

,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4691,2022-11-05,0.0,4.8,18.2,35.0,1.4,4.7,10.0


In [38]:
# Prolongar temperatura del día anterior para los días que no tienen tmin o tmax

df["tmin"] = df["tmin"].fillna(method="ffill")
df["tmax"] = df["tmax"].fillna(method="ffill")

# Show 5 of november 2022
df[df["fecha"] == "2022-11-05"]

C:\Users\Jaime\AppData\Local\Temp\ipykernel_21740\2786834585.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["tmin"] = df["tmin"].fillna(method="ffill")
C:\Users\Jaime\AppData\Local\Temp\ipykernel_21740\2786834585.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["tmax"] = df["tmax"].fillna(method="ffill")


,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4691,2022-11-05,0.0,4.8,18.2,35.0,1.4,4.7,10.0


In [39]:
# Show 11 of march 2022
df[df["fecha"] == "2022-03-11"]

,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4452,2022-03-11,6.8,7.4,13.8,28.0,4.7,12.5,1.3


In [40]:
# Poner a 0 la precipitación de los días que no tienen
df["prec"] = df["prec"].fillna(0)

# Show 11 of march 2022
df[df["fecha"] == "2022-03-11"]

,fecha,prec,tmin,tmax,dir,velmedia,racha,sol
4452,2022-03-11,6.8,7.4,13.8,28.0,4.7,12.5,1.3


## Guardado de los resultados

In [41]:
# Create directory if it doesn't exist
import os
if not os.path.exists(outPath):
    os.makedirs(outPath)

In [42]:
# Save the DataFrame to a csv file
df.to_csv(outPath + f"{estacion}.csv", index=False, encoding="utf-8")